- - -
PACKAGE LOCAL INSTALLATION and USAGE

Editable local installation:
1. Create a virtual environment based on 'environment.yml' file.
2. In the cmd, run: >>> python -m pip install -e "path/to/pyesm"
3. From the virtual environment: >>> import esm
4. Use esm APIs (Model class)


- - - 
MODEL DIRECTORY GENERATION

Generation of a model directory based on a model template or with blank setup files.

In [ ]:
# create model directory with essential setup files
# to be used only to generate models from scratch
# template models can be imported 
import esm

model_dir_name = '4_sut_multi_year_rcot_vb'
main_dir_path = r'c:\Users\baioc\Documents\GitHub\pyesm\default'

esm.create_model_dir(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    default_model=None,
    force_overwrite=False,
)

- - - 
GENERATION OF A NEW MODEL FROM SCRATCH

Generation of a new model defined by setup files.

Step-by-step model creation with sets, data and problems generation.

In [1]:
# generate model instance based on setup files filled by the user.
# validates model directory before executing.
import esm


model_dir_name = '4_sut_multi_year_rcot_vb'
main_dir_path = r'c:\Users\baioc\Documents\GitHub\pyesm\default'
model = esm.Model(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    use_existing_data=False,
)

INFO | Model | 'Model' object initialization...
INFO | Model.file_manager | 'FileManager' object generated.
INFO | Model | Defining settings from model arguments.
INFO | Model | Defining paths from model arguments.
INFO | Model | Model directory and required setup files validated.
INFO | Model.core | 'Core' object initialization...
INFO | Model.core.sql_manager | 'SQLManager' object generation.
INFO | Model.core.index | 'Index' object initialization...
INFO | Model.core.index | Loading and validating data from file, generating 'SetTable' objects.
DEBUG | Model.file_manager | File 'structure_sets.yml' loaded.
INFO | Model.core.index | Loading and validating data from file, generating 'DataTable' objects.
DEBUG | Model.file_manager | File 'structure_variables.yml' loaded.
INFO | Model.core.index | Fetching and validating data, generating 'Variable' objects.
DEBUG | Model.core.index | Fetching 'coordinates_info' to Index.variables.
INFO | Model.core.index | 'Index' object initialized.
INF

In [2]:
# after filling sets.xlsx file: 
# - loading model coordinates to Index
# - generating sqlite database tables for sets and variables
# (variables tables in sqlite database empty)
# (error occurring at the first run related to foreign_keys. second run works)
model.load_model_coordinates()
model.initialize_blank_database()

INFO | Model | Loading new sets data and variable coordinates.
INFO | Model.core.index | 'Index' object: loading new Sets data to Index.sets.
DEBUG | Model.file_manager | Excel file 'sets.xlsx' loaded.
INFO | Model.core.index | 'Index' object: loading variable coordinates to Index.data.
INFO | Model.core.index | 'Index' object: loading variable coordinates to Index.variables.
INFO | Model.core.index | Loading tables 'foreign_keys' to Index.data_tables.
INFO | Model | Generating blank SQLite database and excel input files.
DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
INFO | Model.core.database | Loading Sets to 'database.db'.
DEBUG | Model.core.sql_manager | SQLite table '_set_SCENARIOS' - table overwritten and 1 entries added.
DEBUG | Model.core.sql_manager | SQLite table '_set_SUP_TECH' - table overwritten and 3 entries added.
DEBUG | Model.core.sql_manager | SQLite table '_set_FLOWS' - table overwritten and 3 entries added.
DEBUG | Model.core.sql_manager | SQL

In [3]:
# after filling input_data file/s:
# - loading input data into sqlite database variables tables
# - initialize problem
model.load_data_files_to_database()
model.initialize_problems()

INFO | Model | Loading input data to SQLite database.
DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
INFO | Model.core.database | Loading data input file/s filled by the user to SQLite database.
DEBUG | Model.file_manager | Excel file 'input_data.xlsx' loaded.
DEBUG | Model.core.sql_manager | SQLite table 'd' - 9 entries updated.
DEBUG | Model.core.sql_manager | SQLite table 'u' - 6 entries updated.
DEBUG | Model.core.sql_manager | SQLite table 'c_inv' - 3 entries updated.
DEBUG | Model.core.sql_manager | SQLite table 'c_om' - 3 entries updated.
DEBUG | Model.core.sql_manager | SQLite table 'e' - 3 entries updated.
DEBUG | Model.core.sql_manager | SQLite table 'c_start' - 3 entries updated.
DEBUG | Model.core.sql_manager | SQLite table 'lf_range' - 6 entries updated.
DEBUG | Model.core.sql_manager | SQLite table 'Y' - 10 entries updated.
DEBUG | Model.core.sql_manager | SQLite table 'I_ff' - 6 entries updated.
DEBUG | Model.core.sql_manager | Connection to 'databa

- - - 
MODEL GENERATION FROM EXISTING DATA

Generation of a new model working with existing database and data input files.

In [ ]:
# generate model instance based on setup files filled by the user.
# validates model directory (in case user relies on existing directory). 
# parse settings and paths.
# loading model coordinates
# initializing numerical problem
import esm 

model_dir_name = 
main_dir_path = r'C:\Users\loren\Politecnico di Milano\Documentale DENG - PRIN-MIMO\Models\pyESM_docs\case_studies'

model = esm.Model(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    use_existing_data=True,
    log_level='debug'
)

- - -
DATA and MODEL UPDATE, MODEL RUN 

updating SQLite database with new data, 
re-initializing numerical problem

solving numerical model, 
results export to sqlite database, 
generation of powerbi dataset

In [ ]:
# in case of modifications in input data files (not in sets),
# update database and problem
model.update_database_and_problem()

In [ ]:
# in case of modifications in symbolic problem,
# update problems dataframe and symbolic problem
model.initialize_problems()

In [4]:
# solve numerical problems
model.run_model(verbose=False)

INFO | Model | Running numerical model.
INFO | Model.core.problem | Solving problem: ['baseline'].


Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-30


INFO | Model.core.problem | Problem status: 'optimal'


In [ ]:
model.core.problem.numeric_problems['objective function'][0]

In [5]:
# once model has successfully solved, load endogenous parameters data to 
# sqlite database and generate powerbi dataset.
model.load_results_to_database()
model.generate_pbi_report()

INFO | Model | Exporting endogenous model results to SQLite database.
DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
INFO | Model.core | Fetching data from cvxpy endogenous variables to SQLite database 'database.db' 
DEBUG | Model.core | Fetching data from cvxpy variable 'X' to the related SQLite table.
DEBUG | Model.core.sql_manager | SQLite table 'X' - 15 entries updated.
DEBUG | Model.core | Fetching data from cvxpy variable 'Q' to the related SQLite table.
DEBUG | Model.core.sql_manager | SQLite table 'Q' - 15 entries updated.
DEBUG | Model.core | Fetching data from cvxpy variable 'Q_agg' to the related SQLite table.
DEBUG | Model.core.sql_manager | SQLite table 'Q_agg' - 10 entries updated.
DEBUG | Model.core | Fetching data from cvxpy variable 'C_inv' to the related SQLite table.
DEBUG | Model.core.sql_manager | SQLite table 'C_tot' - 15 entries updated.
DEBUG | Model.core | Fetching data from cvxpy variable 'C_om' to the related SQLite table.
DEBUG | Model.

- - - 
TESTS

In [ ]:
model.core.index.sets.technologies

In [ ]:
model.core.problem.symbolic_problem

In [ ]:
model.core.problem.numeric_problems

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 10
n = 5

# Construct the problem.
x = cp.Variable((n,1))
A = cp.Constant(np.random.randn(m, n))
b = cp.Constant(1)


L = cp.Variable((n,n))
C = cp.Constant(np.random.randn(n, n))
I = cp.Constant(np.eye(n))

objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve(solver='GUROBI', verbose=False)
# The optimal value for x is stored in `x.value`.

prob.status


In [ ]:
# DEFINIZIONE COSTANTI PER TRASFORMARE I VARI OPERATORI VETTORIALI
import cvxpy as cp
import numpy as np

var_a = cp.Variable((3,2))
var_a.value = np.ones((var_a.shape))*3
var_b = cp.Variable((3,1))

i_21 = cp.Constant(np.ones((2,1)))
i_13 = cp.Constant(np.ones((1,3)))

allowed_vars = {
    'A': var_a,
    'B': var_b,
    'i_21': i_21,
    'i_13': i_13,
}

allowed_ops = {
    '+': '+',
    '-': '-',
    '*': '*',
    '@': '@',
    '==': '==',
    '>=': '>=',
    '<=': '<=',
    '(': '(',
    ')': ')',
    ',':',',
    'sum': cp.sum,
}

expr = model.core.problem.execute_cvxpy_code(
    expression='i_13 @ A @ i_21',
    allowed_variables=allowed_vars,
    allowed_operators=allowed_ops
)

expr2 = model.core.problem.execute_cvxpy_code(
    expression='sum(B)',
    allowed_variables=allowed_vars,
    allowed_operators=allowed_ops
)

expr

In [ ]:
import pandas as pd
from esm.util import constants
from esm.util import util

field_id = constants._STD_ID_FIELD['id'][0]

col_to_update = constants._STD_VALUES_FIELD['values'][0]
cols_common = list(variable.coordinates.keys())

new_df = variable.reshaping_variable_data(0)

new_values = {
    's_Name': 'Reference',
    'dt_Name': '2023',
    't_Name': 'Final demand',
    'values': 15.0
}

new_df.loc[len(new_df)] = new_values
new_df.at[0,'values'] = 2

test.core.sqltools.open_connection()
existing_df = test.core.sqltools.table_to_dataframe(table_name=variable.symbol)
existing_df.drop(columns=['id'],inplace=True)
test.core.sqltools.close_connection()

print('\n')
print('existing_df: \n', existing_df)
print('\n')
print('new_df: \n', new_df)

In [ ]:
variable=test.core.index.variables.u

test.core.database.sqltools.open_connection()
raw_data = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name=variable.symbol,
    filters_dict=variable.data['filter'][0]
)
test.core.database.sqltools.close_connection()

raw_data

In [ ]:
test.core.database.sqltools.open_connection()

var = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name='u',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        'f_Name': ['Energy', 'Steel'],
        't_Name': ['PV power plant', 'Steel factory', 'Gas power plant']
    }
)

# filter_1 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='f_Name',
#     parent_table_name='_set_FLOWS',
#     parent_table_fields={
#         'f_Category': ['Product flow'],
#     }
# )

# filter_2 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='dt_Name',
#     parent_table_name='_set_DATETIME',
#     parent_table_fields={
#         'dt_Name': [2023],
#     }
# )

# var_custom_filter = test.core.database.sqltools.filtered_table_to_dataframe(
#     table_name='v',
#     filters_dict={
#         **filter_1,
#         **filter_2,
#         **{'f_Name': ['Steel']},
#     }
# )

test.core.database.sqltools.close_connection()

# filter_1, filter_2
var